In [ ]:
from tkinter import *
from tkinter import messagebox as tkMessageBox
from tkinter import filedialog
import threading
# tkinter library is used for bulding  a GUI window
class gsheets_xbrl:
    # This class is used to parse json data, which is returned from financialmodellingprep api call
    def __init__(self, url):
        self.url = url

    def get_jsonparsed_data(self):
        """
        Receive the content of ``url``, parse it as JSON and return the object.

        Parameters
        ----------
        url : str

        Returns
        -------
        dict
        """
        response = urlopen(self.url)
        data = response.read().decode("utf-8")
        return json.loads(data)


class data_transform:
    # This library is used to clean the data and return arranged dataframes
    def __init__(self, statements, ratios, companyinfo):
        self.statements = statements
        # This statements variable contains data of all the three statements namely; Balance Sheet, Profit and Loss and Cash Flow Statement
        self.ratios = ratios
        # This ratio variable contains data of financial and other ratios of the company
        self.companyinfo = companyinfo
        # This companyinfo variable contains data related to information about the company

    def to_dataframe_statement(self):
        # This function is used to clean the data from financial statements
        statement_data = []
        # Using loop as we have three dataframes
        for t,data in enumerate(self.statements):
            dat = data
            dat = pd.DataFrame(dat)
            dat = dat.drop(columns={'symbol', 'fillingDate', 'acceptedDate', 'link', 'finalLink', 'period'}, axis=1)
            # Above line of code is used to Drop columns from the data frame which are of no use to us
            dat = dat.T.copy()
            header_row = 0
            dat.columns = dat.iloc[header_row]
            dat = dat.reset_index()
            dat = dat.drop([0])
            dat['index'] = dat['index'].str.replace(r"([A-Z])", r" \1").str.strip()
            #Above line of code changes "index" column in the following way: "CashAndCashEquivalents" to "Cash and Cash Equivalents"
            dat = dat[dat.columns[:11][::-1]]
            cols = list(dat.columns)
            cols = [cols[-1]] + cols[:-1]
            dat = dat[cols]
            dat['index'] = dat['index'].str.lower()
            #Above line of code is used to convert column to small alphabets from capital letters
            statement_data.append(dat)
        return statement_data

    def to_dataframe_ratio(self):
        # This function is used to clean data from Ratios
        dat = pd.DataFrame(self.ratios)
        dat = dat.drop(columns={'symbol'}, axis=1)
        dat = dat.T.copy()
        header_row = 0
        dat.columns = dat.iloc[header_row]
        dat = dat.reset_index()
        dat = dat.drop([0])
        dat['index'] = dat['index'].str.replace(r"([A-Z])", r" \1").str.strip()
        dat = dat[dat.columns[:11][::-1]]
        cols = list(dat.columns)
        cols = [cols[-1]] + cols[:-1]
        dat = dat[cols]
        # Above three lines are used to reverse columns into ascending years, leaving the heading column non-reversed
        dat['index'] = dat['index'].str.lower()
        return dat

    def to_dataframe_info(self):
        # This function is used to clean data from company information
        dat = pd.DataFrame(self.companyinfo)
        dat = dat[['companyName', 'symbol', 'mktCap', 'industry', 'description', 'website', 'ceo', 'sector']]
        dat = dat.T
        dat.index = dat.index.str.replace(r"([A-Z])", r" \1").str.strip()
        dat.reset_index(inplace=True)
        dat.rename(columns={0: "Information about company"}, inplace=True)
        dat['index'] = dat['index'].str.lower()
        return dat
import webbrowser
def func(k):
  my_var=str(k+1)
  msg = f'A{my_var}'
  return msg

class google_verify:
    def __init__(self, verify,total_data,email,nsheet):
        self.verify = verify
        self.total_data=total_data
        self.email=email
        self.nsheet=nsheet

    def google_sheets_fill(self):
        # This function is used to create or call previously created google sheets
        gc = pygsheets.authorize(service_file=self.verify)
        try:
            #This try calls the google sheets if present
            createdSpreadsheet= gc.open(f'{self.total_data[0]["Information about company"][0]} Financials')
            # This above line of code tries to open the google sheet if present and if not present moves to except
            createdSpreadsheet.share(self.email, role='writer', type='user')
            worksheets=[]
            for t in range(self.nsheet):
                wks = createdSpreadsheet.worksheet(property='index', value=t)
                wks.clear()
                # This loop is used to clear the sheets
                worksheets.append(wks)
            filling = list(map(lambda x, y: x.set_dataframe(y, start='A1'), worksheets, self.total_data))
            # The above line of code is used to fill the google sheets in all the four sheets namely company information, Balance Sheet,P&L and Ratios
            webbrowser.open(createdSpreadsheet.url, new=2)
            # Above line opens the sheet in web-browser
            return worksheets
        except:
            # This except runs only when the sheet is not present for a particular name
            res = gc.sheet.create(f'{self.total_data[0]["Information about company"][0]} Financials')
            createdSpreadsheet = gc.open_by_key(res['spreadsheetId'])
            createdSpreadsheet.share(self.email, role='writer', type='user')
            sheets_created = ['Company Information', 'Balance Sheet', 'Income Statement', 'Cash Flow Statements',
                              'Ratios']
            worksheets=[]
            for num,sheet in enumerate(sheets_created):
                if sheet=='Company Information':
                    createdSpreadsheet.sheet1.title=sheet
                    wks = createdSpreadsheet.worksheet(property='index', value=num)
                    worksheets.append(wks)
                 # This foor loop creates 4 sheets namely company information, Balance Sheet,P&L and Ratios
                else:
                    createdSpreadsheet.add_worksheet(sheet)
                    wks = createdSpreadsheet.worksheet(property='index', value=num)
                    worksheets.append(wks)
            filling =list( map(lambda x, y: x.set_dataframe(y, start='A1'), worksheets, self.total_data))
            webbrowser.open(res['spreadsheetUrl'], new=2)
            return worksheets
    def color_sheets(self):
        worksheets= self.google_sheets_fill()
        #This above line call the function just above where we create or open google sheets
        balance_terms = 'total current liabilities/total debt/total assets/total liabilities/total current assets'
        income_terms = 'cost of revenue/revenue/gross profit/net income/epsdiluted'
        cash_flow_terms = 'net cash provided by operating activities/net cash used provided by financing activities/net cash used for investing activites/free cash flow/investments in property plant and equipment/acquisitions net'
        terms = [balance_terms, income_terms, cash_flow_terms]
        # This function is used to colour the google sheets or bold the specific things ; like income_terms, balance terms,income_terms and cash_flow_terms mentioned above
        wks = worksheets[0]
        length = len(self.total_data[0])
        t = list(map(func, range(length + 1)))
        k = list(map(lambda x: wks.cell(x).set_text_format('bold', True), t))
        for num, data in enumerate(worksheets[1:4]):
            wks = data
            length = len(self.total_data[num + 1])
            cells = list(map(func, range(length+1)))
            bold_data = list(map(lambda x: wks.cell(x).set_text_format('bold', True) if wks.cell(x).value in terms[num] else None, cells))
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import json


def financial_google(ticker,apikey,filead,email_dat,nsheets_dat=5):
    # This piece of function runs all the functions above namely cleaning data, calling google sheet and making data in google sheet bold
    complete_data=['profile','balance-sheet-statement','income-statement','cash-flow-statement','ratios']
    raw_data=[]
    for num,upload in enumerate(complete_data):
        link=f'https://financialmodelingprep.com/api/v3/{upload}/{ticker}?apikey={apikey}'
        info =gsheets_xbrl(link)
        info=info.get_jsonparsed_data()
        raw_data.append(info)
    complete_data = data_transform(statements=[raw_data[1], raw_data[2], raw_data[3]], ratios=raw_data[4],
                                   companyinfo=raw_data[0])
    financial_statements= complete_data.to_dataframe_statement()
    # This financial_statements variables store all the three financial statements in clean format from the function class cleaning data
    financialratios = complete_data.to_dataframe_ratio()
    # This variable stores clean dataframe of ratios
    company_info= complete_data.to_dataframe_info()
    # This variable  stores data for company information
    complete_data=google_verify(filead,[company_info,financial_statements[0],financial_statements[1],financial_statements[2],financialratios],email_dat,nsheets_dat)
    # This is class call
    complete_data.color_sheets()

import pandas as pd
import pygsheets

class GUI_python:
    # The particular class is used to create the GUI which takes email id, google sheet credentials(upload), API Key and Company Ticker
    def gui_guide(self):
        self.frame = Tk()
        self.frame.resizable(0, 0)
        self.frame.title(" Financial Data for US companies")
        self.frame.geometry('750x200')
        # Above few lines of codes create a window
        self.emailFrame = Frame(self.frame, width=200, height=600, highlightthickness=2,
                                highlightbackground="#111")
        # This creates a frame which creates a box around label which has enter email id text
        self.emailFrame.grid(row=3,column=0, padx=10, pady=2, sticky=N + S)
        self.tickerFrame = Frame(self.frame, width=200, height=300, highlightthickness=2,
                                highlightbackground="#111")
        self.tickerFrame.grid(row=1,column=0, padx=10, pady=2, sticky=N + S)
        self.apiFrame = Frame(self.frame, width=200, height=300, highlightthickness=2,
                                 highlightbackground="#111")
        self.apiFrame.grid(row=2, column=0, padx=10, pady=2, sticky=N + S)
        self.servicFrame = Frame(self.frame, width=200, height=300, highlightthickness=2,
                                  highlightbackground="#111")
        self.servicFrame.grid(row=4, column=0, padx=10, pady=2, sticky=N + S)
        self.l1 = Label(self.tickerFrame, text="Enter a valid Company Ticker:")
        self.l1.grid(row=1, column=0)
        # Above two lines of code create a label with Enter a valid company ticker as text
        self.l2 = Label(self.apiFrame, text="Enter a valid API key:", )
        self.l2.grid(row=2, column=0)
        # Above two lines of code create a label with Enter a valid valid api key as text
        self.l3= Label(self.servicFrame, text="Upload the google API service credentials:")
        self.l3.grid(row=4, column=0)
        self.l4=Label(self.emailFrame, text="Enter your mail id:")
        self.l4.grid(row=3, column=0)
        self.l5= Label(self.frame, text="No File Selected",bg = "white", pady = 5)
        self.l5.grid(row=4, column=1)
        self.E1 = Entry(self.frame, bd=5,)
        self.E1.grid(row=1, column=1)
        self.E2 = Entry(self.frame, bd=5)
        self.E2.grid(row=2, column=1)
        self.E3 = Entry(self.frame, bd=5)
        self.E3.grid(row=3, column=1)
        self.btn2= Button(self.frame, text="Submit", command=self.sci_thread)
        # This is the submit button, when it is clicked on it, the functions sci_thread runs
        self.btn2.grid(row=5, column=0)
        # .grid function is used to position the labels, buttons etc in the GUI window
        self.btn3 = Button(self.frame, text="Close", command=self.frame.destroy)
        self.btn3.grid(row=5, column=1)
        self.btn1 = Button(self.frame, text='Browse', command=self.browse_button)
        self.btn1.grid(row=4, column=2)
        self.l6 = Label(self.frame, text="Click Submit to Update",bg = "white")
        self.l6.grid(row=6,column=0)
        self.frame.mainloop()
    def sci_thread(self):
        maincal = threading.Thread(target=self.proces)
        maincal.start()
        # The GUI which has tkinter hangs if the code behind gui runs much longer, so threading runs this gui and other code on different threads thus avoiding not-responding
    def proces(self):
        # Verify that the data is entered or not before submitting
        try:
            global ticker
            global apikey
            global email
            ticker=str(Entry.get(self.E1))
            apikey=str(Entry.get(self.E2))
            email=str(Entry.get(self.E3))
            text=self.l5.cget("text")
            # When no text is entered in any entry and no file is selected raise this warning
            if ticker=="" or apikey=="" or email==""or text=="No File Selected" or("@" not in email):
                if ticker=="" or apikey=="" or email=="" or "@" not in email:
                    tkMessageBox.showwarning("Warning", "Please enter a valid Ticker, API Key and Email id")
                if "No File" in text:
                    tkMessageBox.showwarning("Warning", "Please select google credentials file")
            else:
                data = financial_google(ticker, apikey, text, email)
                self.l6.config(text="Data Updated")
        except:
            tkMessageBox.showwarning("Warning", "Please enter Valid data")
        # try:
        #     data = financial_google(ticker, apikey, text, email)
        # except:
        #     tkMessageBox.showerror("Error","Incorrect Information Entered")
    def browse_button(self):
        filename = filedialog.askopenfilename(initialdir="/", title="Select file",filetypes=(("json files", "*.json"), ("all files", "*.*")))
        if filename:
            self.l5.config(text=filename)
def main():
    gui = GUI_python()
    tar = gui.gui_guide()
if __name__ == "__main__":
    main()
# gui=GUI_python()
# tar=gui.gui_guide()
# mainfz = threading.Thread(target=gui.gui_guide)
# mainfz.start()



In [ ]:
# You can install python in your computer and try the code, libraries which need to be installed first: pygsheets, pandas,numpy, matplotlib,webbrowser,selenium, urrlib, json
# Install Python from:  https://www.python.org/downloads/release/python-377/
# From here install (scroll down)(for Windows): Windows x86-64 executable installer
# When Installing Python add python to path, follow this link for it: https://datatofish.com/add-python-to-windows-path/
# You can install libraries from command prompt by typing(for example pygsheets):py -m pip install pygsheets  
# Do this for every library
# Then type idle in windows search and paste the code there by creating new file